# Weighted interval scheduling problem

In [1]:
import random
import numpy as np
import pandas as pd

sched_items = pd.DataFrame(index = ["a", "b", "c", "d", "e", "f", "g", "h"])
sched_items["start_time"] = [0, 1, 3, 3, 4, 5, 6, 8]
sched_items["finish_time"] = [6, 4, 5, 8, 7, 9, 10, 11]
sched_items["weight"] = [17, 8, 5, 14, 7, 3, 10, 2]
sched_items

,start_time,finish_time,weight
a,0,6,17
b,1,4,8
c,3,5,5
d,3,8,14
e,4,7,7
f,5,9,3
g,6,10,10
h,8,11,2


In [2]:
def greedy_WISP(gdfItems):
    '''
    For a given a dataframe that holds the information on the items, 
    return total weight of the the scheduled items and the id's of the 
    items that are scheduled (as a list) using earliest finish time first algorithm
    '''

    gdfItems.sort_values("finish_time", ascending=True, inplace=True, kind='mergesort')
    # print(gdfItems)

    t_vListItems = []
    t_vListItems.append(gdfItems.iloc[0].name) # add first item in sorted dataframe
    
    t_vAllItems = list(gdfItems.index)
    t_vAllItems.remove(t_vListItems[0]) # first item is already scheduled, update list
    
    # iterate over all the remaining items
    for j in t_vAllItems:
        if gdfItems.loc[j].start_time >=  gdfItems.loc[t_vListItems[-1]].finish_time:
            t_vListItems.append(gdfItems.loc[j].name)

    tmp_dTotalWeight = gdfItems.loc[t_vListItems].sum().weight
    
    return tmp_dTotalWeight, t_vListItems

In [3]:
dtotalWeight, vlistItems = greedy_WISP(sched_items.copy()) # send the copy so that original df will not be modified
print("dtotalWeight: %s vlistItems: %s" %(dtotalWeight, vlistItems))

dtotalWeight: 17 vlistItems: ['b', 'e', 'h']


In [4]:
def DP_WISP(gdfItems):
    '''
    For a given a dataframe that holds the information on the items, 
    return total weight of the the scheduled items and the id's of the 
    items that are scheduled (as a list) using a DP algorithm
    '''
    
    gdfItems.sort_values("finish_time", ascending=True, inplace=True, kind='mergesort')
    gdfItems["pval"] = 0
    # print(gdfItems)
    # print('\n')

    # gdfItems_new = gdfItems.sort_values("start_time", ascending=True, inplace=False, kind='mergesort')
    # print(gdfItems_new)
    # print('\n')

    # print(list(gdfItems.index)[::-1])

    # calculate p indices
    for i in list(gdfItems.index)[::-1]:
        for j in list(gdfItems.index)[::-1]:
            if i != j:
                if gdfItems.loc[i].start_time >=  gdfItems.loc[j].finish_time:
                    gdfItems.loc[i, "pval"] = j
                    # print(i,j)
                    break
    
    # print(gdfItems)
    # print('\n')

    dic_keys = list(gdfItems.index)
    dic_keys.insert(0,0)
    # print(dic_keys)

    tmp_vListItems = []
    tmp_dpTable = dict(zip(dic_keys, np.zeros(len(dic_keys))))
    # print("tmp_dpTable %s" %tmp_dpTable)
    t_iCtr = 1
    for j in list(gdfItems.index):
        tmp_dpTable[j] = max(gdfItems.loc[j].weight + tmp_dpTable[gdfItems.loc[j].pval], tmp_dpTable[dic_keys[t_iCtr - 1]])
        t_iCtr += 1

    # print("tmp_dpTable %s" %tmp_dpTable)

    dic_keys = list(gdfItems.index)
    dic_keys.insert(0,0)
    dic_vals = np.arange(len(dic_keys))
    # print("dic_keys: %s" %dic_keys)
    # print("dic_vals: %s" %dic_vals)
    my_dict = dict(zip(dic_keys, dic_vals))
    # print("my_dict: %s" %my_dict)

    def find_solution_wisp(j):
        if j == 0:
            return []
        elif (gdfItems.loc[j].weight  + tmp_dpTable[gdfItems.loc[j].pval] > tmp_dpTable[dic_keys[my_dict[j] - 1]]):
            return [j] + find_solution_wisp(gdfItems.loc[j].pval)
        else:
            return find_solution_wisp(dic_keys[my_dict[j] - 1])

    vlistItems = find_solution_wisp(dic_keys[-1])

    # return tmp_dpTable[list(gdfItems.index)[-1]], tmp_dpTable, gdfItems
    return tmp_dpTable[list(gdfItems.index)[-1]], vlistItems

In [5]:
dtotalWeight, vlistItems = DP_WISP(sched_items.copy()) # send the copy so that original df will not be modified
print("dtotalWeight: %s vlistItems: %s" %(dtotalWeight, vlistItems))

dtotalWeight: 27.0 vlistItems: ['g', 'a']


## tests on 5 different cases

In [6]:
for n in range(5):
    sched_items = pd.DataFrame(index = ["a", "b", "c", "d", "e", "f", "g", "h"])
    print("item-ids = %s" %np.array(sched_items.index))
    sched_items["start_time"] = [0, 1, 3, 3, 4, 5, 6, 8]
    sched_items["finish_time"] = [6, 4, 5, 8, 7, 9, 10, 11]

    np.random.seed(n**2)
    sched_items["weight"] = np.random.randint(1, 20, size=len(sched_items["start_time"]))
    print("item-weights = %s" %np.array(sched_items["weight"]))
    
    # print("n: %d" %n)
    # print(sched_items)

    dtotalWeight, vlistItems = greedy_WISP(sched_items.copy()) # send the copy so that original df will not be modified
    print("greedy_WISP: dtotalWeight= %s vlistItems= %s" %(dtotalWeight, vlistItems))

    dtotalWeight, vlistItems = DP_WISP(sched_items.copy()) # send the copy so that original df will not be modified
    print("DP_WISP: dtotalWeight= %s vlistItems= %s" %(dtotalWeight, vlistItems))
    print('\n')

item-ids = ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
item-weights = [13 16  1  4  4  8 10 19]
greedy_WISP: dtotalWeight= 39 vlistItems= ['b', 'e', 'h']
DP_WISP: dtotalWeight= 39.0 vlistItems= ['h', 'e', 'b']


item-ids = ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
item-weights = [ 6 12 13  9 10 12  6 16]
greedy_WISP: dtotalWeight= 38 vlistItems= ['b', 'e', 'h']
DP_WISP: dtotalWeight= 38.0 vlistItems= ['h', 'e', 'b']


item-ids = ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
item-weights = [15  6  2  9  9 19 10  8]
greedy_WISP: dtotalWeight= 23 vlistItems= ['b', 'e', 'h']
DP_WISP: dtotalWeight= 25.0 vlistItems= ['f', 'b']


item-ids = ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
item-weights = [ 2  9 18  2  1 15 11 11]
greedy_WISP: dtotalWeight= 21 vlistItems= ['b', 'e', 'h']
DP_WISP: dtotalWeight= 33.0 vlistItems= ['f', 'c']


item-ids = ['a' 'b' 'c' 'd' 'e' 'f' 'g' 'h']
item-weights = [10 15  6  2  5  5 10  1]
greedy_WISP: dtotalWeight= 21 vlistItems= ['b', 'e', 'h']
DP_WISP: dtotalWeight= 25.0 vlistItems= ['g', 'b']




# Knapsack problem

In [7]:
import random
import numpy as np
import pandas as pd

W = 11

knapsack_items = pd.DataFrame(index = [1,2,3,4,5])
knapsack_items["weight"] = [1, 2, 5, 6, 7]
knapsack_items["value"] = [1, 6, 18, 22, 28]

knapsack_items

,weight,value
1,1,1
2,2,6
3,5,18
4,6,22
5,7,28


In [8]:
def greedy_knapsack(gdfItems, gW):
    '''
    For a given a dataframe that holds the information on the items, 
    return total value of the the items and the id's of the 
    items (as a list) using greedy-by-ratio algo
    '''
    
    gdfItems["ratio"] = np.round(gdfItems["value"]/gdfItems["weight"],2)
    
    gdfItems.sort_values("ratio", ascending=False, inplace=True, kind='mergesort')
    # gdfItems.sort_values("weight", ascending=True, inplace=True, kind='mergesort')

    # print(gdfItems)
    # print('\n')

    t_vSelectedItemIDs = []
    tmp_dVal = 0
    t_dRemaining_cap = gW

    # iterate over all the items
    for i in list(gdfItems.index):
        if(gdfItems.loc[i].weight <=  t_dRemaining_cap):
            tmp_dVal += gdfItems.loc[i].value
            t_vSelectedItemIDs.append(i)
            t_dRemaining_cap = t_dRemaining_cap - gdfItems.loc[i].weight
        
        assert t_dRemaining_cap >=0, "t_dRemaining_cap cannot be smaller than 0"

        if (t_dRemaining_cap == 0):
            break

    return tmp_dVal, t_vSelectedItemIDs

In [9]:
dtotalVal, vlistItems = greedy_knapsack(knapsack_items.copy(), W) # send the copy so that original df will not be modified
print("dtotalVal: %s vlistItems: %s" %(dtotalVal, vlistItems))

dtotalVal: 35.0 vlistItems: [5, 2, 1]


In [10]:
def DP_knapsack(gdfItems, gW):
    '''
    For a given a dataframe that holds the information on the items, 
    return total value of the the items and the id's of the 
    items (as a list) using a DP algo
    '''
    
    t_dpTable = np.zeros((gdfItems.shape[0]+1, gW+1))
    t_dpTable_items = np.zeros((gdfItems.shape[0]+1, gW+1))

    # print(t_dpTable)
    # print('\n')

    for i in list(gdfItems.index):
        for w in range(gW+1):
            if (gdfItems.loc[i].weight <= w) and (gdfItems.loc[i].value + t_dpTable[i-1][w - gdfItems.loc[i].weight] > t_dpTable[i-1][w]):
                t_dpTable[i][w] = gdfItems.loc[i].value + t_dpTable[i-1][w - gdfItems.loc[i].weight]
                t_dpTable_items[i][w] = 1
            else:
                t_dpTable[i][w] = t_dpTable[i-1][w]
    
    # print(t_dpTable)
    # print('\n')

    # print(t_dpTable_items)
    # print('\n')

    tmp_vpicks = []
    t_dRemaining_cap = gW

    # traverse the DP table to identify which items were picked
    for i in range(gdfItems.shape[0],0,-1):
        if t_dpTable_items[i][t_dRemaining_cap] == 1:
            tmp_vpicks.append(i)
            t_dRemaining_cap -= gdfItems.loc[i].weight
    tmp_vpicks.sort()
    # print("tmp_vpicks: %s" %tmp_vpicks)

    return t_dpTable[list(gdfItems.index)[-1]][gW], tmp_vpicks

In [11]:
dtotalVal_DP, vlistItems_DP = DP_knapsack(knapsack_items.copy(), W)
print("dtotalVal_DP: %s vlistItems_DP: %s" %(dtotalVal_DP, vlistItems_DP))

dtotalVal_DP: 40.0 vlistItems_DP: [3, 4]


## test on 5 different cases

In [12]:
for n in range(5):
    W = 11

    knapsack_items = pd.DataFrame(index = [1,2,3,4,5])
    knapsack_items["weight"] = [1, 2, 5, 6, 7]
    print("item-weights = %s" %np.array(knapsack_items["weight"]))

    np.random.seed(n**3)
    knapsack_items["value"] = np.random.randint(1, 42, size=len(knapsack_items["weight"]))
    print("item-values = %s" %np.array(knapsack_items["value"]))

    dtotalVal, vlistItems = greedy_knapsack(knapsack_items.copy(), W) # send the copy so that original df will not be modified
    print("greedy_knapsack: dtotalVal= %s vlistItems= %s" %(dtotalVal, vlistItems))

    dtotalVal_DP, vlistItems_DP = DP_knapsack(knapsack_items.copy(), W)
    print("DP_knapsack: dtotalVal_DP= %s vlistItems_DP= %s" %(dtotalVal_DP, vlistItems_DP))
    print('\n')

item-weights = [1 2 5 6 7]
item-values = [ 1  4  4 40 10]
greedy_knapsack: dtotalVal= 45.0 vlistItems= [4, 2, 1]
DP_knapsack: dtotalVal_DP= 45.0 vlistItems_DP= [1, 2, 4]


item-weights = [1 2 5 6 7]
item-values = [38 13  9 10 12]
greedy_knapsack: dtotalVal= 60.0 vlistItems= [1, 2, 3]
DP_knapsack: dtotalVal_DP= 63.0 vlistItems_DP= [1, 2, 5]


item-weights = [1 2 5 6 7]
item-values = [ 4 21  6 27  9]
greedy_knapsack: dtotalVal= 52.0 vlistItems= [2, 4, 1]
DP_knapsack: dtotalVal_DP= 52.0 vlistItems_DP= [1, 2, 4]


item-weights = [1 2 5 6 7]
item-values = [20  9 32 38 25]
greedy_knapsack: dtotalVal= 61.0 vlistItems= [1, 3, 2]
DP_knapsack: dtotalVal_DP= 70.0 vlistItems_DP= [3, 4]


item-weights = [1 2 5 6 7]
item-values = [ 5 39 39 22 32]
greedy_knapsack: dtotalVal= 83.0 vlistItems= [2, 3, 1]
DP_knapsack: dtotalVal_DP= 83.0 vlistItems_DP= [1, 2, 3]


